# ナップサック(その1)


[atcoder](https://atcoder.jp/contests/dp/tasks/dp_d)より

```
N 個の品物があります。 品物には 1,2,…,N と番号が振られています。 
各 i (1≤i≤N) について、品物 i の重さは wi で、価値は vi です。
太郎君は、N 個の品物のうちいくつかを選び、ナップサックに入れて持ち帰ることにしました。 
ナップサックの容量は W であり、持ち帰る品物の重さの総和は W 以下でなければなりません。
太郎君が持ち帰る品物の価値の総和の最大値を求めてください。
```

## とりあえず手始めに再帰(その1)を使ってとく

- 良く考えると全部の組み合わせを探索しているので計算時間はめちゃ遅いO(n**2)???

In [8]:
n, sum_w = map(lambda x: int(x), input().split())

# goods = list[]
all_weights = []
all_values = []
for _ in range(n):  
    w, v = map(lambda x: int(x), input().split())
    all_weights.append(w)
    all_values.append(v)
    
def get_max_value(weights, values, max_sum_weight):
    if (len(weights) <= 0) or (max_sum_weight < min(weights)):
        return 0

    max_value = 0
    for i in range(len(weights)):
        if weights[i] <= max_sum_weight:
            remaining_weights = weights[:i] + weights[i+1:]
            remaining_values = values[:i] + values[i+1:]
            max_value = max(max_value, values[i] + get_max_value(remaining_weights, remaining_values, max_sum_weight - weights[i]))
    return max_value

print(get_max_value(all_weights, all_values, sum_w))



1 1
1 20
20


## 再帰(その2)を使ってとく

- メモ化を使いやすいように
- **現在の注目している品物の番号**と、現時点でのナップサックの残り容量を関数の引数にしてみるとうまくいきます。 再帰関数をrec(i, j)とし、「品物i～n-1を重さjのナップサックに入れたときの価値の最大値」を返す関数を表すことにします。
- 計算時間はまだマシ???O(logn)とか???

In [41]:
n, sum_w = map(lambda x: int(x), input().split())

all_weights = []
all_values = []
for _ in range(n):  
    w, v = map(lambda x: int(x), input().split())
    all_weights.append(w)
    all_values.append(v)
    
memo = {}
    

def max_value(i_item, remaining_weight):
    if (i_item, remaining_weight) in memo.keys():
        return memo[(i_item, remaining_weight)]
    
    if i_item == n - 1:
        memo[(i_item, remaining_weight)] = all_values[i_item] if (all_weights[i_item] <= remaining_weight) else 0
        return memo[(i_item, remaining_weight)]
    
    if all_weights[i_item] > remaining_weight:
        memo[(i_item, remaining_weight)] = max_value(i_item+1, remaining_weight)
        return memo[(i_item, remaining_weight)]
    
    memo[(i_item, remaining_weight)] = max(all_values[i_item] + max_value(i_item+1, remaining_weight - all_weights[i_item]),
        max_value(i_item+1, remaining_weight))
    return memo[(i_item, remaining_weight)]

print(max_value(0, sum_w))

6 15
6 5
5 6
6 4
6 6
3 5
7 2
17


それでもTLEする。

## DPを使ってとく


縦軸に`何個めまでの品物を使ったか`、横軸に`0-weightまで`


それぞれのますに、横軸のweightを超えない重さで最高のvalueを埋めていく


肝となるの考え方は、`dp[i][max_w] = max(dp[i-1][max_w], dp[i-1][max_w-current_w]+current_w)`

In [57]:
n, sum_w = map(lambda x: int(x), input().split())

dp = [0]*(sum_w+1)
for _ in range(n):  
    w, v = map(lambda x: int(x), input().split())
    # weightは大きい方から更新していく必要がある
    for max_w in range(sum_w, w-1, -1):
        dp[max_w] = max(dp[max_w], v + dp[max_w-w])
#     print(dp)
print(dp[sum_w])

6 15
6 5
5 6
6 4
6 6
3 5
7 2
17


まだTLEする